In [ ]:
from faker import Faker
import random
from datetime import datetime, timedelta
import numpy as np

# Function to generate random chief complaint
def generate_chief_complaint(age):
    chief_complaints = {
        'Child': ['Fever', 'Cough', 'Earache', 'Sore throat'],
        'Adult': ['Chest pain', 'Shortness of breath', 'Abdominal pain', 'Headache']
    }
    if age < 18:
        return random.choice(chief_complaints['Child'])
    else:
        return random.choice(chief_complaints['Adult'])

# Function to generate random patient encounters
def generate_patient_encounters(patient_data):
    encounters = []
    for patient in patient_data:
        num_encounters = random.randint(1, 5)  # Generate random number of encounters
        for _ in range(num_encounters):
            dob = datetime.strptime(patient['DateOfBirth'], '%Y-%m-%d')
            start_date = dob + timedelta(days=365 * 2)  # 2 years after birth
            end_date = datetime.today()
            encounter_date = fake.date_time_between_dates(datetime_start=start_date, datetime_end=end_date)
            chief_complaint = generate_chief_complaint(patient['Age'])
            encounter = {
                'PatientName': patient['Name'],
                'MedicalRecordNumber': patient['MedicalRecordNumber'],
                'ChiefComplaint': chief_complaint,
                'EncounterDate': encounter_date.replace(tzinfo=None).strftime('%Y-%m-%d %H:%M:%S')
            }
            encounters.append(encounter)
    return encounters

encounters_data = generate_patient_encounters(patient_data)

# Printing generated encounters data for demonstration
print("Encounters Data:")
for encounter in encounters_data:
    print(encounter)